In [4]:
import pandas as pd
import matplotlib as plt
import numpy as np
%matplotlib inline

In [5]:
filter_source = ['All Source', 'Other Biomass', 'Other', 'Other Gases', 'Ohter Biomass', 'Wood and Wood Derived Fuels', 'Geothermal', 'Nuclear', 'Pumped Storage']
wanted_source = ['Coal', 'Petroleum', 'Hydroelectric', 'Natural Gas', 'Wind', 'Solar Thermal and Photovoltaic']
statelist = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 
             'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 
             'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [6]:
plants = pd.read_csv('plantcapacity.csv')
plants = plants.drop(['Producer Type','Generators','Nameplate Capacity (Megawatts)','Summer Capacity (Megawatts)'],axis = 1)
plants = plants[plants['Year'].isin([2015])] 

for item in filter_source:
    plants = plants[plants['Fuel Source'].str.contains(item) == False]

plants = plants.rename(columns = {'State Code':'State','Fuel Source':'Source'})    
plants.head()

,Year,State,Source,Facilities
37306,2015,AK,Coal,1
37308,2015,AK,Coal,2
37310,2015,AK,Petroleum,3
37312,2015,AK,Coal,2
37313,2015,AK,Hydroelectric,30


In [7]:
mod_plants = []
for state in statelist:
    state_plants = plants[plants['State'].isin([state])]
    state_plants['Facilities'] = state_plants['Facilities'].astype(int)
    state_plants = state_plants.groupby(['Year', 'State', 'Source'], as_index = False).agg(sum)
    mod_plants.append(state_plants)
mod_plants = pd.concat(mod_plants, axis=0).reset_index(drop = True)
mod_plants.head()

/Users/IvanC/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Year,State,Source,Facilities
0,2015,AK,Coal,10
1,2015,AK,Hydroelectric,60
2,2015,AK,Natural Gas,30
3,2015,AK,Petroleum,192
4,2015,AK,Wind,12


In [17]:
final_plants = pd.DataFrame()
final_plants['State'] = statelist
final_plants.insert(0, 'Year', 2015)

for item in wanted_source:
    source_df = mod_plants[mod_plants['Source'].isin([item])]
    source_df = source_df.drop(['Year', 'Source'], axis = 1)
    source_df = source_df.rename(columns = {'Facilities':item}) 
    final_plants = pd.merge(final_plants, source_df, on = 'State', how = 'left').fillna(0)
    final_plants[item] = final_plants[item].astype(int)
    

final_plants = final_plants.rename(columns = {'Petroleum':'Petro','Natural Gas':'NG','Hydroelectric':'Hydro','Solar Thermal and Photovoltaic':'Solar'})
final_plants = final_plants[['Year', 'State', 'Coal', 'NG', 'Petro', 'Hydro', 'Solar', 'Wind']]
final_plants = final_plants[final_plants['State'] != 'DC']
final_plants

,Year,State,Coal,NG,Petro,Hydro,Solar,Wind
0,2015,AK,10,30,192,60,0,12
1,2015,AL,14,54,8,48,0,0
2,2015,AR,10,36,6,38,2,0
3,2015,AZ,12,62,8,20,118,10
4,2015,CA,4,580,30,502,742,230
5,2015,CO,26,62,26,84,58,48
6,2015,CT,2,68,56,28,6,0
8,2015,DE,2,20,10,0,14,2
9,2015,FL,26,142,42,4,26,0
10,2015,GA,14,68,52,60,36,0


In [19]:
total_e = pd.read_csv('arranged_esource')
total_e = total_e.drop(['Unnamed: 0', 'Nuclear', 'Other Biomass', 'Other Gases', 'Pumped Storage', 'Total', 'Wood and Wood Derived Fuels', 'Geothermal', 'Other'],axis = 1)
total_e = total_e.rename(columns = {'Petroleum':'Petro','Natural Gas':'NG','Hydroelectric Conventional':'Hydro','Solar Thermal and Photovoltaic':'Solar'})
total_e = total_e[['Year', 'State', 'Coal', 'NG', 'Petro', 'Hydro', 'Solar', 'Wind']]
total_e = total_e[total_e['Year'].isin([2015])].reset_index(drop = True)
total_e

,Year,State,Coal,NG,Petro,Hydro,Solar,Wind
0,2015,AK,1335099,6179426,1493134,3137858,0,319388
1,2015,AL,82820859,111688913,168416,19723608,0,0
2,2015,AR,43480066,29732572,120888,7138390,1854,0
3,2015,AZ,72334338,67314205,96236,13071444,6913216,904070
4,2015,CA,595748,232279218,169030,27616996,29628768,24459118
5,2015,CO,63081789,23287499,14530,3240322,502742,14950558
6,2015,CT,1200938,34724618,873583,604934,34878,0
7,2015,DC,0,45306,196,0,0,0
8,2015,DE,1197176,13378535,308267,0,97023,10252
9,2015,FL,85808018,311648304,4299030,488588,447866,0


In [16]:
average_plant_capacity = pd.DataFrame()
average_plant_capacity[final_plants.columns[0:2]] = final_plants.iloc[:,0:2]
average_plant_capacity[final_plants.columns[2:]] = total_e.iloc[:,2:].div(final_plants.iloc[:,2:],axis = 0).fillna(0).astype(int)
average_plant_capacity = average_plant_capacity.replace(-9223372036854775808, 0)
average_plant_capacity = average_plant_capacity[average_plant_capacity['State'] != 'DC']
average_plant_capacity

,Year,State,Coal,NG,Petro,Hydro,Solar,Wind
0,2015,AK,133509,205980,7776,52297,0,26615
1,2015,AL,5915775,2068313,21052,410908,0,0
2,2015,AR,4348006,825904,20148,187852,927,0
3,2015,AZ,6027861,1085712,12029,653572,58586,90407
4,2015,CA,148937,400481,5634,55013,39930,106343
5,2015,CO,2426222,375604,558,38575,8667,311469
6,2015,CT,600469,510656,15599,21604,5813,0
8,2015,DE,598588,668926,30826,0,6930,5126
9,2015,FL,3300308,2194706,102357,122147,17225,0
10,2015,GA,5304375,1484390,10014,99452,7191,0


In [20]:
final_plants.to_csv('final_plants.csv')
average_plant_capacity.to_csv('average_plant_capacity.csv')